In [160]:
import sys
sys.path.append('C:/Anaconda3/Lib/site-packages')

import sqlite3
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import traceback
%load_ext snakeviz

## European Football Kaggle Database
Author: Alex Galea
- +25,000 matches
- +10,000 players
- 11 European Countries with their lead championship
- Seasons 2008 to 2016
- Player's statistics sourced from EA Sports' FIFA video game series, including the weekly updates
- Team line up with squad formation (X, Y coordinates)
- Betting odds from up to 10 providers
- Detailed match events (goal types, possession, corner, cross, fouls, cards etc...) for +10,000 matches
---
I would like to develop an algorithm for identifying "good bets" where the better is likely to beat the odds. This is a supervised learning problem where the target is probably a binary classification (win/loss or over/under).

The idea here is not to try and predict the winner, but instead guess probabilities and identify situations where you can "beat the odds". For example: for a given team and season, compute the probabilities of winning the bet for each game and compare to the odds. Then look at the cases where there is the largest difference between predicted probabilites and odds.

In [3]:
conn = sqlite3.connect('my_database.sqlite')
c = conn.cursor()

#### Team ID dictionaries

In [4]:
# ID key exists in the "Team" table
c.execute('PRAGMA TABLE_INFO(Team)')
c.fetchall()

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'team_api_id', 'INTEGER', 0, None, 0),
 (2, 'team_long_name', 'TEXT', 0, None, 0),
 (3, 'team_short_name', 'TEXT', 0, None, 0)]

In [161]:
%%snakeviz

Team_long, Team_short = {}, {}
row_id = 0
c.execute('SELECT MAX(id) FROM Team')
max_id = c.fetchall()[0][0]
while True:
    try:
        c.execute('SELECT team_api_id, team_long_name, team_short_name FROM Team WHERE id="{}"'.\
                  format(row_id))
        query = c.fetchall()[0]
        Team_long[query[0]] = query[1]
        Team_short[query[0]] = query[2]
    except:
        pass
    if row_id == max_id:
        print('Final row reached (number {0:s})'.format(str(row_id)))
        break
    row_id += 1

Final row reached (number 51538)
 
*** Profile stats marshalled to file 'C:\\Users\\agale\\AppData\\Local\\Temp\\tmpd1tos14p'. 


In [6]:
for i in list(Team_long.keys())[:10]:
    print(Team_long[i], Team_short[i])
print('...')

Fortuna Duesseldorf FDU
FC Zuerich ZUR
Leicester LEI
Montpellier MON
KV Mechelen MEC
Aston Villa AVL
Troyes TRO
Ajax AJA
Wolfsburg WOL
FC Cologne FCK
...


#### Useful features?
This may involve a lot of data collection on my part.
- teams
    - include ELO score (points at that given time in the season) as a fraction (i.e. home_ELO_score/away_ELO_score). when extracting will have to assign ELO score of 1 if 0 (at start of season) to aviod dividing by zero errors with ratio
    - home or away
    - goals on the year
- players
    - cumulative player stats (i.e. total career goals)
- meta
    - betting odds?? since I'm ultimately comparing to the betting odds, is it cheating if I also include the average betting odds across all companies as a feature?

*the Elo system is widely used by sharp bettors as a reliable tool in the search of value bets, i.e. bets with positive expected value, because it provides a solid statistical foundation for calculating the probabilities of the different outcomes with great accuracy.

___

Let's start with just the FIFA EA player data (which has time dependence!) and generate features for each game.

- use average value of all players on the team
    - break it down by position so that each has different weights?
- do something else for binary features

#### Add EA Sports player data to Match table

In [42]:
# c.execute('SELECT MAX(id) FROM Match')
# max_id = c.fetchall()[0][0]

# List of row ids for Match table
c.execute('SELECT id FROM Match')
Match_id = [f[0] for f in sorted(c.fetchall(), key=lambda x: x[0])]
len(Match_id)

25945

In [41]:
# Perentage of column that has an entry

def isMatchElement(column, row_id):
    c.execute('SELECT {0:s} FROM Match WHERE id="{1:d}"'.\
              format(column, row_id))
    query = c.fetchall()[0]
    if not query[0]:
        return False
    else:
        return True

def fullFrac(column, Match_id):
    column_isnul = [isMatchElement(column, row_id) for row_id in Match_id]
    return np.sum(column_isnul)/len(column_isnul)

full_frac = fullFrac('id', Match_id)
print(full_frac)

full_frac = fullFrac('goal', Match_id)
print(full_frac)

full_frac = fullFrac('home_player_1', Match_id)
print(full_frac)

1.0
0.0
0.952823280015


What Y values are we dealing with and how can we determine forwards vs defense ..etc.

In [71]:
cols = ", ".join(["away_player_Y"+str(i) for i in range(1,12)])
c.execute('SELECT {0:s} FROM Match'.\
         format(", ".join(["away_player_Y"+str(i) for i in range(1,12)])))
aa = c.fetchall()

bb = np.array([a for row in aa for a in row])
from collections import Counter
Counter(bb)

Counter({0: 22,
         1: 24113,
         3: 95306,
         5: 2892,
         6: 15787,
         7: 58435,
         8: 23311,
         9: 2738,
         10: 31029,
         11: 11632,
         None: 20130})

With motivation to keep the distribution as even as possible, we have:  
1 - goalie  
3 - defence  
5, 6, 7 - mid  
8, 9, 10, 11 - forward  

Will have to be careful with the incomplete ones. For example we have some as follows:

In [74]:
[row for row in aa if row[0]==0]

[(0, 0, None, None, None, None, None, None, None, None, None),
 (0, 0, None, None, None, None, None, None, None, None, None),
 (0, 0, None, None, None, None, None, None, None, None, None),
 (0, 0, None, None, None, None, None, None, None, None, None),
 (0, 0, None, None, None, None, None, None, None, None, None),
 (0, 0, None, None, None, None, None, None, None, None, None),
 (0, 0, None, None, None, None, None, None, None, None, None),
 (0, 0, None, None, None, None, None, None, None, None, None),
 (0, 0, None, None, None, None, None, None, None, None, None),
 (0, 0, None, None, None, None, None, None, None, None, None),
 (0, 0, None, None, None, None, None, None, None, None, None)]

In [144]:
print([row for row in aa if row[0]==None][:10])
print('...')

[(None, None, None, None, None, None, None, None, None, None, None), (None, None, None, None, None, None, None, None, None, None, None), (None, None, None, None, None, None, None, None, None, None, None), (None, None, None, None, None, None, None, None, None, None, None), (None, None, None, None, None, None, None, None, None, None, None), (None, None, None, None, None, None, None, None, None, None, None), (None, None, None, None, None, None, None, None, None, None, None), (None, None, None, None, None, None, None, None, None, None, None), (None, None, None, None, None, None, None, None, None, None, None), (None, None, None, None, None, None, None, None, None, None, None)]
Ellipsis


In [157]:
def getTeamScores(match_id, team, EA_stats,
                  group='forward_mid_defense_goalie'):
    ''' Return the cumulative average team scores for 
    a given EA Sports FIFA statistic.
    team : str
        'home' or 'away'
    EA_stat : dict
        Names of statistics to cumulate for goalie and players.
        e.g. {'player': 'overall_rating, heading_accuracy',
              'goalie': 'gk_diving, gk_handling'}
    group : str
        How to group scores:
        'forward_mid_defense_goalie': output 4 values
        'all': output 1 value (currently not implemented)
    '''
    
    if team == 'home':
        player_cols = ', '.join(['home_player_'+str(i) for i in range(1,12)])
        player_Y_cols = ', '.join(['home_player_Y'+str(i) for i in range(1,12)])
    elif team == 'away':
        player_cols = ', '.join(['away_player_'+str(i) for i in range(1,12)])
        player_Y_cols = ', '.join(['away_player_Y'+str(i) for i in range(1,12)])
        
    # Get the player ids from the Match table
    c.execute('SELECT {0:s} FROM Match WHERE id={1:d}'.\
             format(player_cols, match_id))
    player_api_id = np.array(c.fetchall()[0])
    
    # Exit if all items in the list are null
    if False not in [p==0 or p==None for p in player_api_id]:
        raise LookupError('Found null EA stats entry at match_id={}'.\
                                   format(match_id))
    
    # Get the player positions from the Match table
    # We only care about the Y position to designate
    # forwards, midfielders, defense, and goalie
    c.execute('SELECT {0:s} FROM Match WHERE id={1:d}'.\
             format(player_Y_cols, match_id))
    player_Y = c.fetchall()[0]
    def givePosition(Y):
        if Y == 1:
            return 'G'
        elif Y == 3:
            return 'D'
        elif Y == 5 or Y == 6 or Y == 7:
            return 'M'
        elif Y == 8 or Y == 9 or Y == 10 or Y == 11:
            return 'F'
        else:
            sys.exit('Unknown value for Y: {}'.\
                    format(Y))
    player_pos = np.array([givePosition(Y) for Y in player_Y])
    
    # Get the match date
    def toDatetime(datetime):
        return dt.datetime.strptime(datetime, '%Y-%m-%d %H:%M:%S')
    c.execute('SELECT date FROM Match WHERE id={}'.\
             format(match_id))
    match_date = toDatetime(c.fetchall()[0][0])
    
    # Lookup the EA Sports stats for each player
    # The stats are time dependent so we have to
    # find the ones closest to the match date
    def getBestDate(player_id, match_date):
        c.execute('SELECT id FROM Player_Stats WHERE player_api_id={}'.\
                 format(player_id))
        ids = np.array([i[0] for i in c.fetchall()])
        c.execute('SELECT date_stat FROM Player_Stats WHERE player_api_id={}'.\
                 format(player_id))
        dates = [toDatetime(d[0]) for d in c.fetchall()]
        dates_delta = np.array([abs(d-match_date) for d in dates])
        return ids[dates_delta==dates_delta.min()][0]
    positions = ('G', 'D', 'M', 'F')
    average_stats = {}
    for position in positions:
        print(position)
        if position == 'G':
            stats = EA_stats['goalie']
        else:
            stats = EA_stats['player']
        position_ids = player_api_id[player_pos==position]
        average_stats[position] = np.zeros(len(stats.split(',')))
        for player_id in position_ids:
            print(player_id)
            best_date_id = getBestDate(player_id, match_date)
            c.execute('SELECT {0:s} FROM Player_Stats WHERE id={1:d}'.\
                     format(stats, best_date_id))
            query = np.array(c.fetchall()[0])
            print(query)
            print(average_stats[position])
            if sum([q==None for q in query]):
                raise LookupError('Found null EA stats entry at stat_id={}'.\
                                  format(best_date_id))
#                 sys.exit('Found null EA stats entry at stat_id={}'.\
#                         format(best_date_id))
            average_stats[position] += query
            print('')
        average_stats[position] /= len(position_ids) # take average

    return average_stats
            
            
EA_stats = {'player': ', '.join(['overall_rating', #'attacking_work_rate', 'defensive_work_rate',
                                  'crossing', 'finishing', 'heading_accuracy', 'short_passing',
                                  'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
                                  'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
                                  'agility', 'reactions', 'balance', 'shot_power', 'jumping',
                                  'stamina', 'strength', 'long_shots', 'interceptions',
                                  'positioning', 'vision', 'penalties', 'marking',
                                  'standing_tackle', 'sliding_tackle']),
            'goalie': ', '.join(['gk_diving', 'gk_handling', 'gk_kicking',
                                 'gk_positioning', 'gk_reflexes'])}

def print_exception():
    print(str(e))
    with open('errors.out', 'w') as f:
        f.write(str(e)); f.write('\n')
    ex_type, ex, tb = sys.exc_info()
    traceback.print_tb(tb)

# for i in np.arange(1, 1000, 100):
if True:
    i=401
    try:
        getTeamScores(i, 'home', EA_stats)
    except LookupError as e:
        print('LookupError')
        print_exception()
    except Exception as e:
        print_exception()

G
38289
[61 62 53 65 63]
[ 0.  0.  0.  0.  0.]

D
38292
[67 51 32 67 54 None 31 None 31 46 54 56 55 None 64 None 60 None 62 67 30
 78 82 None 80 65 72 None]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
LookupError
Found null EA stats entry at stat_id=135847


  File "<ipython-input-157-156781fcc063>", line 116, in <module>
    getTeamScores(i, 'home', EA_stats)
  File "<ipython-input-157-156781fcc063>", line 84, in getTeamScores
    raise LookupError('Found null EA stats entry at stat_id={}'.                                  format(best_date_id))


In [ ]:
try:
    getTeamScores(1, 'home', EA_stats)
except LookupError as e:
    print('LookupError')
    print_exception()
except Exception as e:
    print_exception()

Exceptions are raised from rows with ALL null elements. If we encoutner a row with some Null values we'll deal with it. There are many cases like this, and in particular at the start of the database index.

In [149]:
# for i in np.arange(1, 1000, 100):
if True:
    i=401
    try:
        getTeamScores(i, 'home', EA_stats)
    except LookupError as e:
        print('LookupError')
        print_exception()
    except Exception as e:
        print_exception()

G
38289
operands could not be broadcast together with shapes (4,) (5,) (4,) 


  File "<ipython-input-149-e8cd4ae57199>", line 5, in <module>
    getTeamScores(i, 'home', EA_stats)
  File "<ipython-input-148-fea005edc4c4>", line 84, in getTeamScores
    average_stats[position] += query


#### The model?
- Logistic regression one vs all?
    - not good with mixed data (some discrete (categorical) and some continuous)
- Decision trees?
    - Andrew insists I use boosted trees (XGBoost)